# Transforma em DataFrame e limpa

In [1]:
import pandas as pd
from functions.df_helpers import replace_decimal_character
from services.read_data import read_tabular
from services.save_data import save_df

Leitura do DataFrame

In [2]:
# Lê excel e transforma em DataFrame
sales_df = read_tabular('silver', 'sales', '2025', '11')

Remove e renomeia colunas

In [3]:
# Cria tupla de sufixos de colunas necessários
required_suffixes = tuple(['Vendas', 'Anúncios', 'Publicidade'])

# Define lista de colunas do dataframe e remove 'Reclamação_encerrada_Reclamações' da lista pois é uma exceção a regra definida e será necessária
df_columns = list(sales_df.columns)
df_columns.remove('Reclamação_encerrada_Reclamações')

# Remove colunas desnecessárias pelo sufixo
for col in df_columns:
    if not col.endswith(required_suffixes):
        sales_df = sales_df.drop(col, axis=1)

# Dropa mais colunas desnecessárias
columns_to_drop = ['Descrição_do_status_Vendas', 'Pacote_de_diversos_produtos_Vendas', 'Pertence_a_um_kit_Vendas', 'SKU_Anúncios', '#_de_anúncio_Anúncios', 'Canal_de_venda_Anúncios', 'Receita_por_acréscimo_no_preço_(pago_pelo_comprador)_Vendas', 'Taxa_de_parcelamento_equivalente_ao_acréscimo_Vendas']
sales_df = sales_df.drop(columns_to_drop, axis=1)

In [4]:
# Renomeia as colunas
new_names = ['id_venda', 'data_venda', 'status_venda', 'unidades_vendidas', 'receita_por_produto', 'tarifas_impostos_venda', 'receita_envio', 'tarifa_envio', 'custo_envio', 'diff_custo_envio', 'cancelamentos_reembolsos', 'total_vendas', 'mes_faturamento_tarifas', 'venda_publicidade', 'titulo_anuncio', 'variacao_anuncio', 'preco_unitario_anuncio', 'tipo_anuncio', 'reclamacao_encerrada']
sales_df.columns = new_names

Filtra apenas vendas concluídas

In [5]:
import numpy as np

# Cria lista auxiliar que crirá coluna count_produto
count_product = []

# Para cada linha do df, checa se existe uma condição que deve anular a contagem do produto
# Como todas as linhas são necessárias para análise, cria uma coluna 'count_produto' e marca com False nesses casos
for row in sales_df.itertuples():
    if (row.total_vendas <= 0) or ('cancel' in row.status_venda.lower()) or ((row.reclamacao_encerrada == 1) & (np.isnan(row.total_vendas))):
        count_product.append(False)
    else:
        count_product.append(True)

# Cria coluna count_produto
sales_df['count_produto'] = count_product

Corrige tipos e valores

In [6]:
# Converte o id de int para string
sales_df['id_venda'] = sales_df['id_venda'].astype(str)

In [7]:
# Remove a hora da coluna de data
sales_df['data_venda'] = sales_df['data_venda'].str[:-9]

In [8]:
# Muda tipo das colunas para int
sales_df[['unidades_vendidas', 'reclamacao_encerrada']] = sales_df[['unidades_vendidas', 'reclamacao_encerrada']].fillna(0).astype(int)

# Substitui ponto por virgula nas colunas decimais
sales_df = replace_decimal_character(sales_df)

In [9]:
# Remove os espaços vazios da coluna de variação do anuncio e as chaves 'cor' e 'tamanho'
sales_df['variacao_anuncio'] = sales_df['variacao_anuncio'] \
                                    .str.replace(' ', '', regex=False) \
                                    .str.replace('Cor:', '', regex=False) \
                                    .str.replace('Tamanho:', '', regex=False)

In [10]:
# Cria as colunas cor kimono e tamanho kimono a partir da coluna variação anúncio
sales_df[['cor_kimono', 'tamanho_kimono']] = sales_df['variacao_anuncio'].str.split('|', expand=True)

In [12]:
# Salva DataFrame
save_df(sales_df, 'gold', 'sales_final', '2025', '11', 'sales_final')